# Working in AE5 with Multiple Envs
## Basics
- Anaconda enterprise launches a pod with 4 containers:
  - two with the editor image, one for the actual editor the other for synching repo
  - two with proxy image, one for each of the above
- the ae-editor image, is huge in docker terms, it is the current mechnisim we adopted to deliver the pre-canned environments to all the cluster node (this will be handled by the container runtime and will be cached on all hosts, if configured with high performence storage, that will provide better perormence.
- the canned environments are:

In [9]:
!conda env list | awk '!/#/ {print $1}'

base
anaconda201908_py37
anaconda44_py27
anaconda44_py35
anaconda44_py36
anaconda44_r
anaconda50_hadoop
anaconda50_impyla
anaconda50_py27
anaconda50_py35
anaconda50_py36
anaconda50_sas
lab_launch
py36
sync_launch



The official environments for 5.4.0 are as follow...
**anaconda201908_py37** - this should probably be used for most cases     
anaconda44_py27          
anaconda44_py35         
anaconda44_py36          
anaconda44_r             
anaconda50_hadoop        
anaconda50_impyla       
anaconda50_py27          
anaconda50_py35         
anaconda50_py36         
anaconda50_sas          

- environments used by AE5 - not for developing projects
base               
lab_launch                              
sync_launch    

- base is where conda is installed
- lab_launch is where the editor environment is installed (Jupyter notebook/lab zeppelin, etc...) as well as all the anaconda-project and other supporting tools for preparing the session.
- base is in base /opt/continuum/anaconda 
- all other envs are /opt/continuum/anaconda/envs/<env-name>
- lab_launch has many binaries linked from base (like conda)

ANACONDA_PROJECT_ENVS_PATH is set so that new environment created by anaconda-project are created there, and not where the command is executed

In [2]:
!env | grep ANACONDA_PROJECT_ENVS_PATH

ANACONDA_PROJECT_ENVS_PATH=/opt/continuum/anaconda/envs


In [ ]:
!conda env list

When a session is launched, the first thing that will happen is the first environment is being prepared. The environments and their pakages are listed in anaconda-project.yml and you can have a many as you like. However they will not be prepared (only the first will).

You can prepare the environment from the command line with anaconda-project prepare --env-spec and the name of the env. or you could update anaconda-project to the latest version and prepare all environments with --all. In either case you have to do that from the terminal and from the lab_launch environment (not from the notebook).
An easy way to simplify this, is to include a standard set of scripts in your project that can simplify tasks this - like I have here in *scripts/prep.sh*

```
conda update anaconda-project -y && anaconda-project prepare --all
```

Notice that if you include envspec in anaconda-project yaml with no packages - as in ```anaconda201908_py37: {}``` for example, since this is one of the prebuilt environment, the startup time will be very fast. you can then add packages to the env if they are missing. If you specify the exact version you would like the preperation will be faster.

With multiple environment in anaconda-project.yml and with the environments prepared, you can click on any of the preparation ticks in the ***Project tab*** and that will add and register those environments in jupyter lab (added as kernels and configured in launcher). Note that this step is mandathey will not appear without this step.


Next you may notice that the launcher does not select meaningful names for the kernels being used, you can rename the kernels to the environment name. I do this with a small function in **scripts/prep.sh** you can take a look at that to see how that works, the Project jupyter_lab extension registers and configuer launchers using all the kernelspecs it finds in the prepared environments (only the anaconda-project.yml ones). it also configures a terminal with the specific environment ac